In [45]:
!pip install -qU langchain \
langchain_openai \
langchain-community \
langchain_core \
langchain_huggingface \
langgraph \
langchain_qdrant \
qdrant_client \
arxiv \
pymupdf \
tiktoken \
sentence_transformers

In [46]:
import getpass
import os
from google.colab import userdata

# OPENAI KEY
# Try to get API key from Colab secrets
api_key = userdata.get("OPENAI_API_KEY")
# If not found, prompt the user
if not api_key:
    api_key = getpass.getpass("Enter your OpenAI API key: ")
# Set it as an environment variable
os.environ["OPENAI_API_KEY"] = api_key

# TAVILY API KEY
tavily_api_key = userdata.get("TAVILY_API_KEY")
if not tavily_api_key:
    tavily_api_key = getpass.getpass("Enter your TAVILY API key: ")
os.environ["TAVILY_API_KEY"] = tavily_api_key

#LANGCHAIN
langchain_api_key = userdata.get("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
if not langchain_api_key:
    langchain_api_key = getpass.getpass("Enter your LANGCHAIN API key: ")
os.environ["LANGCHAIN_API_KEY"] = langchain_api_key
os.environ["LANGCHAIN_PROJECT"] = f"AIE6 - LangGraph"

In [47]:
import os
# import chainlit as cl  # importing chainlit for our app
from dotenv import load_dotenv
from typing import TypedDict, Annotated, List
from typing_extensions import List, TypedDict
from dotenv import load_dotenv
import operator
from pathlib import Path
from tqdm import tqdm
from langchain.document_loaders import PyMuPDFLoader
from langchain.prompts import ChatPromptTemplate
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.tools.arxiv.tool import ArxivQueryRun
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.documents import Document
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_qdrant import QdrantVectorStore
from langgraph.graph import START, StateGraph, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
import tiktoken

In [48]:
from langchain.document_loaders import PyMuPDFLoader

docs = PyMuPDFLoader("https://www.medicare.gov/publications/10050-medicare-and-you.pdf").load()

In [63]:
import tiktoken
from langchain.text_splitter import RecursiveCharacterTextSplitter

def tiktoken_len(text):
    tokens = tiktoken.encoding_for_model("gpt-4o-mini").encode(
        text,
    )
    return len(tokens)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 100,
    length_function = tiktoken_len,
)

split_chunks = text_splitter.split_documents(docs)

In [64]:
len(split_chunks)

128

In [65]:
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
# from sentence_transformers import SentenceTransformer

# embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
# embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
embeddings = HuggingFaceEmbeddings(model_name="vivnatan/snowflake-arctic-embed-l-medicare")

# model = SentenceTransformer("vivnatan/snowflake-arctic-embed-l-medicare")


client = QdrantClient(":memory:")

client.create_collection(
    collection_name="medicare",
    # vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
    vectors_config=VectorParams(size=1024, distance=Distance.COSINE),
)

vector_store = QdrantVectorStore(
    client=client,
    collection_name="medicare",
    embedding=embeddings,
)

_ = vector_store.add_documents(documents=split_chunks)

retriever = vector_store.as_retriever(search_kwargs={"k": 5})

Some weights of BertModel were not initialized from the model checkpoint at vivnatan/snowflake-arctic-embed-l-medicare and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# def retrieve(state):
#   retrieved_docs = retriever.invoke(state["question"])
#   return {"context" : retrieved_docs}

def retrieve(state):
  retrieved_docs = retriever.invoke(state["question"])
  if not retrieved_docs:
      return {"context": [], "response": "I don't know"}  # Set fallback response directly
  return {"context": retrieved_docs}

In [ ]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
You are a seasoned health insurance advisor with over 30 years in the industry.
You have tremendous knowledge about the industry and the different players.
You keep track of the changes in regulations and you are here to advise the user with their questions or concerns.
If there are any industry jargons in your responses make them easy to understand and provide details of what they are.
Wherever available provide references.
If you are unable to get the answer from the context provided, just say that you don't know. Don't try to make up an answer.

### Question
{question}

### Context
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

In [ ]:
from langchain_openai import ChatOpenAI

openai_chat_model = ChatOpenAI(model="gpt-4o-mini")

llm = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
# def generate(state):
#   docs_content = "\n\n".join(doc.page_content for doc in state["context"])
#   messages = rag_prompt.format_messages(question=state["question"], context=docs_content)
#   response = llm.invoke(messages)
#   return {"response" : response.content}

def generate(state):
  # If we already set a fallback in retrieve
  if "response" in state and state["response"] == "I don't know":
      return {"response": "I don't know"}

  docs_content = "\n\n".join(doc.page_content for doc in state["context"])
  messages = rag_prompt.format_messages(question=state["question"], context=docs_content)
  response = llm.invoke(messages)
  return {"response" : response.content}

In [ ]:
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain_core.documents import Document

class State(TypedDict):
  question: str
  context: List[Document]
  response: str

In [ ]:
# graph_builder = StateGraph(State).add_sequence([retrieve, generate])
# graph_builder.add_edge(START, "retrieve")
# graph = graph_builder.compile()

graph_builder = StateGraph(State)
graph_builder.add_node("retrieve", retrieve)
graph_builder.add_node("generate", generate)
graph_builder.set_entry_point("retrieve")

In [ ]:
# Conditional: only call generate if context exists
def has_context(state):
    if not state.get("context"):
        return END
    return "generate"

In [ ]:
graph_builder.add_conditional_edges("retrieve", has_context)
graph_builder.add_edge("generate", END)

graph = graph_builder.compile()

In [ ]:
#/*@tool
# def ai_rag_tool(question: str) -> str:
#   """Useful for when you need to answer any question. Start here. Input should be a fully formed question."""
#   response = graph.invoke({"question" : question})
#   return {
#         "messages": [HumanMessage(content=response["response"])],
#         "context": response["context"]
#     }

@tool
def ai_rag_tool(question: str) -> dict:
  """Useful for when you need to answer questions about Medicare.
  Input should be a fully formed question."""
  response = graph.invoke({"question" : question})

  if not response["context"] or response["response"].strip().lower() == "i don't know":
      return {
          "messages": [HumanMessage(content="I don't know the answer to that question based on the available information.")],
          "context": []
      }

  return {
      "messages": [HumanMessage(content=response["response"])],
      "context": response["context"]
  }

In [ ]:
# Initialize the tools
tavily_tool = TavilySearchResults(max_results=5)
arxiv_tool = ArxivQueryRun()

# Assemble the tool belt
tool_belt = [
    # tavily_tool,
    arxiv_tool,
    ai_rag_tool
]

In [ ]:
tool_belt

In [ ]:
model = ChatOpenAI(model="gpt-4o", temperature=0)
model = model.bind_tools(tool_belt)

In [ ]:
class AgentState(TypedDict):
  messages: Annotated[list, add_messages]
  context: List[Document]

In [ ]:
tool_node = ToolNode(tool_belt)

# def call_tool(state: AgentState) -> AgentState:
#     new_state = tool_node.invoke(state)
#     return new_state

In [ ]:
uncompiled_graph = StateGraph(AgentState)

In [ ]:
# def call_model(state):
#   messages = state["messages"]
#   response = model.invoke(messages)
#   return {
#         "messages": [response],
#         "context": state.get("context", [])
#     }

def call_model(state):
    messages = state["messages"]
    last_msg = messages[-1]

    # Don't short-circuit on first human message
    if isinstance(last_msg, HumanMessage):
        response = model.invoke(messages)
        return {
            "messages": [response],
            "context": state.get("context", [])
        }

    # If model has no tool calls (didn't call ai_rag_tool), assume it didn't find a tool to run
    if isinstance(last_msg, AIMessage) and not last_msg.tool_calls:
        return {
            "messages": [HumanMessage(content="I don't know the answer to that question based on the available information.")],
            "context": []
        }

    response = model.invoke(messages)
    return {
        "messages": [response],
        "context": state.get("context", [])
    }


In [ ]:
from langgraph.graph import StateGraph, END

uncompiled_graph.add_node("agent", call_model)
uncompiled_graph.add_node("action", tool_node)

In [ ]:
uncompiled_graph.set_entry_point("agent")

In [ ]:
# def should_continue(state):
#   last_message = state["messages"][-1]

#   if last_message.tool_calls:
#     return "action"

#   return END
from langchain.schema import AIMessage

def should_continue(state):
  last_message = state["messages"][-1]

  if isinstance(last_message, AIMessage) and last_message.tool_calls:
    return "action"

  return END

uncompiled_graph.add_conditional_edges(
    "agent",
    should_continue
)


In [ ]:
uncompiled_graph.add_edge("action", "agent")

In [ ]:
compiled_graph = uncompiled_graph.compile()

In [ ]:
compiled_graph

In [ ]:
from langchain_core.messages import HumanMessage

# inputs = {"messages" : [HumanMessage(content="How do I enroll for Medicare online? What are the steps?")]}
inputs = {"messages" : [HumanMessage(content="What is COBRA?")]}

async for chunk in compiled_graph.astream(inputs, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        print(values["messages"])
        print("\n\n")

In [ ]:
# Receiving update from node: 'agent'
# [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_WYgwOL3jyudtSI9UyF8yq4CF', 'function': {'arguments': '{"question":"How does COBRA work with Medicare?"}', 'name': 'ai_rag_tool'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 139, 'total_tokens': 163, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_d8864f8b6b', 'id': 'chatcmpl-BWpv3uKCe59LxcFIiPw6X32GTUjdM', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--3325f85f-9361-4ba9-9e8b-8a9b12176670-0', tool_calls=[{'name': 'ai_rag_tool', 'args': {'question': 'How does COBRA work with Medicare?'}, 'id': 'call_WYgwOL3jyudtSI9UyF8yq4CF', 'type': 'tool_call'}], usage_metadata={'input_tokens': 139, 'output_tokens': 24, 'total_tokens': 163, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]



# Receiving update from node: 'action'
# [ToolMessage(content='{\'messages\': [HumanMessage(content="COBRA (Consolidated Omnibus Budget Reconciliation Act) allows individuals to temporarily continue their employer or union health coverage after employment ends or if they lose coverage as a dependent of a covered employee. If you are eligible for Medicare and you choose COBRA, it is important to know a few key points:\\n\\n1. **COBRA and Medicare Costs**: COBRA coverage may only pay a small portion of your medical costs when you are also eligible for Medicare. You may have to pay most of the health costs out-of-pocket. It is advised to contact your COBRA plan to inquire about the specific percentage they cover.\\n\\n2. **Medicare Enrollment**: If you\'re considering COBRA, it might be beneficial to sign up for Medicare (especially Part B) right away, to avoid unexpected medical bills. You have a specific time frame—8 months after your employment-based coverage ends—to enroll in Part B without incurring a late enrollment penalty.\\n\\n3. **Special Enrollment Period for Part D**: If your COBRA coverage includes creditable prescription drug coverage, you will have a Special Enrollment Period to enroll in Medicare drug coverage (Part D) after your COBRA coverage ends without facing a penalty.\\n\\n4. **Special Enrollment Periods**: It\'s important to note that COBRA is not considered coverage based on current employment, so you generally don\'t qualify for a Special Enrollment Period to sign up for Medicare when your COBRA ends.\\n\\nFor personalized assistance, it is recommended to reach out to your State Health Insurance Assistance Program (SHIP) or contact the Benefits Coordination & Recovery Center at the provided phone numbers.\\n\\nIf you have further questions about how to transition from COBRA to Medicare, reaching out to a trusted agent or broker may also help clarify your options.", additional_kwargs={}, response_metadata={})], \'context\': [Document(metadata={\'producer\': \'Adobe PDF Library 17.0\', \'creator\': \'Adobe InDesign 20.1 (Macintosh)\', \'creationdate\': \'2025-01-22T13:47:26-05:00\', \'source\': \'https://www.medicare.gov/publications/10050-medicare-and-you.pdf\', \'file_path\': \'https://www.medicare.gov/publications/10050-medicare-and-you.pdf\', \'total_pages\': 128, \'format\': \'PDF 1.6\', \'title\': \'Medicare and You Handbook 2025\', \'author\': \'Centers for Medicare and Medicaid Services\', \'subject\': \'Medicare and You Handbook 2025\', \'keywords\': \'Medicare and You Handbook 2025; National version\', \'moddate\': \'2025-02-14T14:29:43-05:00\', \'trapped\': \'\', \'modDate\': "D:20250214142943-05\'00\'", \'creationDate\': "D:20250122134726-05\'00\'", \'page\': 88, \'_id\': \'3bff590e11c24e60a7571cee0adbe1ae\', \'_collection_name\': \'ai_across_years\'}, page_content=\'89\\nSection 6: Medicare drug coverage (Part D)\\n89\\nCOBRA (Consolidated Omnibus Budget Reconciliation Act)\\nThis federal law may allow you to temporarily keep employer or union health \\ncoverage after the employment ends or after you lose coverage as a dependent \\nof the covered employee. There may be reasons why you should take Part B \\ninstead of, or in addition to, COBRA coverage (page 18). However, if you take \\nCOBRA and you’re eligible for Medicare, COBRA may only pay a small portion \\nof your medical costs, and you may have to pay most of the costs yourself. \\nContact your COBRA plan and ask what percent they pay. To avoid unexpected \\nmedical bills, you may need to sign up for Medicare right away. Talk with your \\nState Health Insurance Assistance Program (SHIP) for free, personalized help \\nwith this decision. Go to pages 114–117 for the phone number of your local SHIP.\\nIf you have COBRA that includes creditable prescription drug coverage, \\nyou’ll have a Special Enrollment Period to get Medicare drug coverage \\n(Part D) without paying a penalty when the COBRA coverage ends. If you \\nhave questions about Medicare and COBRA, call the Benefits Coordination \\n& Recovery Center at 1-855-798-2627. TTY users can call 1-855-797-2627. A \\ntrusted agent or broker may also be able to help.\\nMedicare Supplement Insurance (Medigap) with drug coverage\\nMedigap policies can no longer be sold with drug coverage, but if you have an \\nolder Medigap policy that was sold with drug coverage, you can keep it. You \\nmay choose to join a separate Medicare drug plan because most Medigap drug \\ncoverage isn’t creditable, and you may pay more if you join a drug plan later. \\nGo to page 83. \\nYou can’t have drug coverage in both Medigap and your Medicare drug plan. If \\nyou decide to join a separate Medicare drug plan, tell your Medigap insurance \\ncompany so they can remove the drug coverage and adjust your premiums. \\nCall your Medigap insurance company for more information.\\nHow does other government insurance work with \\nMedicare drug coverage (Part D)?\'), Document(metadata={\'producer\': \'Adobe PDF Library 17.0\', \'creator\': \'Adobe InDesign 20.1 (Macintosh)\', \'creationdate\': \'2025-01-22T13:47:26-05:00\', \'source\': \'https://www.medicare.gov/publications/10050-medicare-and-you.pdf\', \'file_path\': \'https://www.medicare.gov/publications/10050-medicare-and-you.pdf\', \'total_pages\': 128, \'format\': \'PDF 1.6\', \'title\': \'Medicare and You Handbook 2025\', \'author\': \'Centers for Medicare and Medicaid Services\', \'subject\': \'Medicare and You Handbook 2025\', \'keywords\': \'Medicare and You Handbook 2025; National version\', \'moddate\': \'2025-02-14T14:29:43-05:00\', \'trapped\': \'\', \'modDate\': "D:20250214142943-05\'00\'", \'creationDate\': "D:20250122134726-05\'00\'", \'page\': 17, \'_id\': \'fdf6add421904ea895abc39bb4a732b4\', \'_collection_name\': \'ai_across_years\'}, page_content=\'18\\nSection 1: Signing up for Medicare\\n18\\nImportant! COBRA (Consolidated Omnibus Budget Reconciliation Act) \\ncoverage isn’t considered coverage based on current employment and \\ndoesn’t count as employer coverage for a Special Enrollment Period. \\nThe same is true for retiree health plans, VA coverage, and individual \\nhealth insurance coverage (like coverage through the Health Insurance \\nMarketplace®). If you’re considering COBRA, there may be reasons why \\nyou should take Part B instead of, or in addition to, COBRA coverage. You \\nhave 8 months after your coverage based on current employment ends to \\nsign up for Part B without a penalty, whether or not you choose COBRA. \\nHowever, if you have COBRA and you’re eligible for Medicare, COBRA \\nmay only pay a small portion of your medical costs. You generally aren’t \\neligible for a Special Enrollment Period to sign up for Medicare when that \\nCOBRA coverage ends. Go to page 89 for more information about COBRA \\ncoverage. To avoid paying a penalty, make sure you sign up for Medicare \\nwhen you’re first eligible. If you have retiree coverage, it may not pay for \\nyour health services if you don’t have both Part A and Part B.\\nExceptional situations for a Special Enrollment Period\\nThere are other circumstances where you may be able to sign up for Medicare \\nduring a Special Enrollment Period. You may be eligible if you miss an \\nenrollment period because of certain exceptional circumstances, like being \\nimpacted by a natural disaster or an emergency, incarceration, employer or \\nhealth plan error, losing Medicaid coverage, or other circumstances outside \\nof your control that Medicare determines to be exceptional. For more \\ninformation, visit Medicare.gov or call 1-800-MEDICARE (1-800-633-4227). \\nTTY users can call 1-877-486-2048.\\nImportant! If you recently lost Medicaid and you now qualify for \\nMedicare, but didn’t sign up for Medicare when you first became \\neligible, you may be able to sign up for Part A and Part B without \\npaying a late enrollment penalty. If you already have Medicare but \\nlost Medicaid, you also have coverage options. For more information,\'), Document(metadata={\'producer\': \'Adobe PDF Library 17.0\', \'creator\': \'Adobe InDesign 20.1 (Macintosh)\', \'creationdate\': \'2025-01-22T13:47:26-05:00\', \'source\': \'https://www.medicare.gov/publications/10050-medicare-and-you.pdf\', \'file_path\': \'https://www.medicare.gov/publications/10050-medicare-and-you.pdf\', \'total_pages\': 128, \'format\': \'PDF 1.6\', \'title\': \'Medicare and You Handbook 2025\', \'author\': \'Centers for Medicare and Medicaid Services\', \'subject\': \'Medicare and You Handbook 2025\', \'keywords\': \'Medicare and You Handbook 2025; National version\', \'moddate\': \'2025-02-14T14:29:43-05:00\', \'trapped\': \'\', \'modDate\': "D:20250214142943-05\'00\'", \'creationDate\': "D:20250122134726-05\'00\'", \'page\': 109, \'_id\': \'d52e497085b64a48a97b9dce117996fe\', \'_collection_name\': \'ai_across_years\'}, page_content=\'110 Section 9: Find helpful contacts and more information\\n110\\nMedicare is working to better coordinate your care\\nMedicare continues to look for ways to better coordinate your care and to \\nmake sure that you get the best health care possible. \\nHere are examples of how your health care providers can better coordinate \\nyour care: \\nAccountable Care Organizations\\nAn Accountable Care Organization (ACO) is a group of doctors, hospitals, and \\nother health care providers who accept Original Medicare and work together \\nto coordinate your health care.\\nWorking as part of an ACO helps your doctors and other health care \\nproviders understand your health history and talk to one another about \\nyour care and your health care needs. This may save you time, money, \\nand frustration by avoiding repeated tests and appointments. More \\ncoordination also helps prevent medical errors and unexpected drug \\ninteractions that may happen if one provider isn’t aware of what another \\nhas prescribed you.\\nImportant! An ACO won’t limit your choice of health care providers. If \\nyour doctor or other provider is part of an ACO, you still have the right \\nto visit any doctor, hospital, or other provider that takes Medicare at \\nany time. \\nIn addition, if your primary care doctor participates in an ACO, you may be \\nable to get more benefits. For example, in some ACOs, your provider may offer \\nmore telehealth services. This means you may be able to get some services \\nfrom home using technology, like your phone or a computer, to communicate \\nin real time with your health care provider. \\nIn addition, a doctor or other provider who is part of an ACO may be able to \\nsend their patients for skilled nursing facility care or rehabilitation services \\neven if they haven’t stayed in a hospital for 3 days first, which is usually a \\nrequirement in Medicare. For you to qualify for this benefit, your doctor or \\nother provider has to decide that you need skilled nursing facility care and \\nmeet certain other eligibility requirements.\\nIf your primary care doctor participates in an ACO and you have Original \\nMedicare, you’ll get a written notice and/or find a poster in their office about \\ntheir ACO participation. There are now hundreds of ACOs across the country.\'), Document(metadata={\'producer\': \'Adobe PDF Library 17.0\', \'creator\': \'Adobe InDesign 20.1 (Macintosh)\', \'creationdate\': \'2025-01-22T13:47:26-05:00\', \'source\': \'https://www.medicare.gov/publications/10050-medicare-and-you.pdf\', \'file_path\': \'https://www.medicare.gov/publications/10050-medicare-and-you.pdf\', \'total_pages\': 128, \'format\': \'PDF 1.6\', \'title\': \'Medicare and You Handbook 2025\', \'author\': \'Centers for Medicare and Medicaid Services\', \'subject\': \'Medicare and You Handbook 2025\', \'keywords\': \'Medicare and You Handbook 2025; National version\', \'moddate\': \'2025-02-14T14:29:43-05:00\', \'trapped\': \'\', \'modDate\': "D:20250214142943-05\'00\'", \'creationDate\': "D:20250122134726-05\'00\'", \'page\': 118, \'_id\': \'6c1aae0759784a2c8ad17709a50f6865\', \'_collection_name\': \'ai_across_years\'}, page_content=\'119\\n119\\nSection 10:  \\nDefinitions \\nAccountable Care Organization (ACO)\\nGroups of doctors, hospitals, and other health care professionals working \\ntogether to give you high-quality, coordinated service and health care.\\nAssignment\\nAn agreement by your doctor, provider, or supplier to be paid directly by \\nMedicare, to accept the payment amount Medicare approves for the service, \\nand not to bill you for any more than the Medicare deductible and coinsurance. \\nBenefit period\\nThe way that Original Medicare measures your use of hospital and skilled \\nnursing facility services. A benefit period begins the day you’re admitted as \\nan inpatient in a hospital or skilled nursing facility. The benefit period ends \\nwhen you haven’t gotten any inpatient hospital care (or skilled care in a skilled \\nnursing facility) for 60 days in a row. If you go into a hospital or a skilled \\nnursing facility after one benefit period has ended, a new benefit period \\nbegins. You must pay the inpatient hospital deductible for each benefit period. \\nThere’s no limit to the number of benefit periods. \\nCoinsurance\\nAn amount you may be required to pay as your share of the cost for benefits \\nafter you pay any deductibles. Coinsurance is usually a percentage (for \\nexample, 20%). \\nCopayment\\nAn amount you may be required to pay as your share of the cost for benefits \\nafter you pay any deductibles. A copayment is a fixed amount, like $30.\\nCreditable prescription drug coverage\\nPrescription drug coverage that’s expected to pay, on average, at least as \\nmuch as Medicare drug coverage. This could include drug coverage from a \\ncurrent or former employer or union, TRICARE, Indian Health Service, VA, or \\nindividual health insurance coverage.\\nCritical access hospital\\nA small facility located in a rural area more than 35 miles (or 15 miles if in \\nmountainous terrain or in areas with only secondary roads) from another \\nhospital or critical access hospital. This facility provides 24/7 emergency care, \\nhas 25 or fewer inpatient beds, and maintains an average length of stay of 96 \\nhours or less for acute care patients.\'), Document(metadata={\'producer\': \'Adobe PDF Library 17.0\', \'creator\': \'Adobe InDesign 20.1 (Macintosh)\', \'creationdate\': \'2025-01-22T13:47:26-05:00\', \'source\': \'https://www.medicare.gov/publications/10050-medicare-and-you.pdf\', \'file_path\': \'https://www.medicare.gov/publications/10050-medicare-and-you.pdf\', \'total_pages\': 128, \'format\': \'PDF 1.6\', \'title\': \'Medicare and You Handbook 2025\', \'author\': \'Centers for Medicare and Medicaid Services\', \'subject\': \'Medicare and You Handbook 2025\', \'keywords\': \'Medicare and You Handbook 2025; National version\', \'moddate\': \'2025-02-14T14:29:43-05:00\', \'trapped\': \'\', \'modDate\': "D:20250214142943-05\'00\'", \'creationDate\': "D:20250122134726-05\'00\'", \'page\': 34, \'_id\': \'4eae65de885842d0aff22e21facd4810\', \'_collection_name\': \'ai_across_years\'}, page_content=\'35\\nSection 2: Find out what Medicare covers\\n35\\nNote: If you’re in a Medicare Advantage Plan, Original Medicare may cover \\nsome costs along with your Medicare Advantage Plan. Contact your plan for \\ndetails about coverage for clinical research studies.\\nCognitive assessment & care plan services\\nWhen you visit your provider (including your yearly “Wellness” visit), they \\nmay perform a cognitive assessment to look for signs of dementia, including \\nAlzheimer’s disease. Signs of cognitive impairment include trouble remembering, \\nlearning new things, concentrating, managing finances, or making decisions. \\nConditions like depression, anxiety, and delirium can also cause confusion, so it’s \\nimportant to understand why you may be having symptoms.\\nMedicare covers a separate visit with a doctor or health care provider to do a full \\nreview of your cognitive function, establish or confirm a diagnosis like dementia \\nor Alzheimer’s disease, and develop a care plan. You can bring someone with \\nyou, like a spouse, friend, or caregiver, to help provide information and answer \\nquestions.\\nDuring this visit, the doctor or health care provider may:\\n•\\t Perform an exam, talk with you about your medical history, and review your \\nmedications.\\n•\\t Identify your social supports including care that your usual caregiver can \\nprovide. \\n•\\t Create a care plan to help address and manage your symptoms.\\n•\\t Help you develop or update your advance care plan. Go to pages 30–31.\\n•\\t Refer you to a specialist, if needed.\\n•\\t Help you understand more about community resources, like rehabilitation \\nservices, adult day health programs, and support groups. \\nThe Part B deductible and coinsurance apply.\\nSome people living with dementia and their family and unpaid caregivers may \\nbe able to get additional support through the “Guiding an Improved Dementia \\nExperience Model” pilot program. Talk to your provider for more information \\nand to find out if they participate.\\nPreventive service  \\nColorectal cancer screenings \\nMedicare covers these screenings to help find precancerous growths or find \\ncancer early, when treatment is most effective. Medicare may cover one or more \\nof these screening tests: \\n•\\t Computed tomography (CT) colonography: Medicare covers this screening\')]}', name='ai_rag_tool', id='fc01c5c0-8f59-4366-a2a5-cdc45e6d3290', tool_call_id='call_WYgwOL3jyudtSI9UyF8yq4CF')]



# Receiving update from node: 'agent'
# [AIMessage(content="COBRA (Consolidated Omnibus Budget Reconciliation Act) allows individuals to temporarily continue their employer or union health coverage after employment ends or if they lose coverage as a dependent of a covered employee. Here's how COBRA works with Medicare:\n\n1. **COBRA and Medicare Costs**: If you are eligible for Medicare and choose COBRA, COBRA may only cover a small portion of your medical costs. You might have to pay most of the health costs out-of-pocket. It's advisable to contact your COBRA plan to understand the specific percentage they cover.\n\n2. **Medicare Enrollment**: It's beneficial to sign up for Medicare (especially Part B) immediately to avoid unexpected medical bills. You have an 8-month window after your employment-based coverage ends to enroll in Part B without a late enrollment penalty.\n\n3. **Special Enrollment Period for Part D**: If your COBRA coverage includes creditable prescription drug coverage, you will have a Special Enrollment Period to enroll in Medicare drug coverage (Part D) after your COBRA coverage ends without facing a penalty.\n\n4. **Special Enrollment Periods**: COBRA is not considered coverage based on current employment, so you generally don't qualify for a Special Enrollment Period to sign up for Medicare when your COBRA ends.\n\nFor personalized assistance, you can reach out to your State Health Insurance Assistance Program (SHIP) or contact the Benefits Coordination & Recovery Center.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 284, 'prompt_tokens': 4375, 'total_tokens': 4659, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_d8864f8b6b', 'id': 'chatcmpl-BWpvDaEZEmjwUy74ucgy4zk0fvgno', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--53e89613-5a11-460b-bb15-c887d3f534d4-0', usage_metadata={'input_tokens': 4375, 'output_tokens': 284, 'total_tokens': 4659, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]
